# Evidence for ecological niche partitioning among ribbon and spotted

seals in the Bering Sea and implications for their resiliency to climate change

Josh M. London [](https://orcid.org/0000-0002-3647-5046) (Alaska Fisheries Science Center)  
March 22, 2024

Ecoclogial niche partitioning allows sympatric species to co-exist in space and time while reducing direct competition for resources. In deep-diving seals (*Phocidae*) such niche partitioning can be observed as temporal differentiation in overlap, two-dimensional separation in space, three-dimensional separation in the water column, and variability in isotopic space. Here, we focus on two species of phocid seals in Bering Sea of Alaska – ribbon seals (*Histriophoca fasciata*) and spotted seals (*Phoca largha*) – and evidence for niche partitioning from two decades of bio-logger deployments and sampling of stable isotopes from multiple tissue types with different intergration times.

In [ ]:
library(RPostgres)
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'dbplyr'

The following objects are masked from 'package:dplyr':

    ident, sql

Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE


Attaching package: 'ctmm'

The following object is masked from 'package:ggplot2':

    annotate

# Introduction

# Methods

In [ ]:
locs_sf <- readRDS(here::here('data/locs_sf.rds'))

A total of 234 bio-loggers (SPLASH family, Wildlife Computers, Redmond, Washington, USA) were deployed on 110 ribbon seals and 82 spotted seals between 2005 and 2022. The deployments span all age classes with the exception of dependent pups for both species and were deployed during the months of April, May, and June. In some cases, deployments were initiated prior to molting and the bio-loggers fell off after a period of weeks to two months. Deployments initiated after molting transmitted up to 9 months.

All deployments were checked for any data quality issues and inconsistent location estimates before they were run through a course speed filter to remove any locations that would have required a sustained swim speed greater than 25 km/h.

Two data sets were created to separate movement in the months of April, May, and June from the rest of the year.

In [ ]:
filter_tracks <- function(tracks_sf) {
  
  crs <- sf::st_crs(tracks_sf)
  dat <- tracks_sf %>%
    sf::st_transform(4326) %>%
    ungroup() %>%
    arrange(deployid, locs_dt)
  dat_tr <- trip::trip(dat, c("locs_dt","deployid"), correct_all = FALSE)
  
  suppressWarnings(
    keep <- trip::sda(
      dat_tr,
      smax = 25 #km/hour
    )
  )
  
  tracks_filt <- dat %>%
    mutate(sda_keep = keep) %>%
    filter(sda_keep) %>%
    dplyr::select(-c(sda_keep, rank)) %>%
    st_transform(crs)
  return(tracks_filt)
}

locs_sf <- locs_sf |> 
  filter_tracks()

locs_sf_spring <- locs_sf |> 
  dplyr::filter(tag_type %in% c('SPLA','SGPS')) |> 
  dplyr::filter(lubridate::month(locs_dt) %in% c(3,4,5)) |> 
  dplyr::arrange(deployid,locs_dt) 

locs_sf_open_water <- locs_sf |> 
  dplyr::filter(tag_type %in% c('SPLA','SGPS')) |> 
  dplyr::filter(!lubridate::month(locs_dt) %in% c(3,4,5)) |> 
  dplyr::arrange(deployid,locs_dt) 

Utilization distributions were estimated for each species and each of the seasonal periods based on a continuous time movement model (R package `ctmm`). Specifically, optimally weighted auto-correlated kernel density estimates (wAKDE) were created to reflect a more honest account of space use while also mitigating sampling bias from irregular deployment lengths.

In [ ]:
as_telem <- function(locs_sf, out_proj) {
  locs_df <- locs_sf |> 
    sfheaders::sf_to_df(fill = TRUE) |> 
    rename(
      individual.local.identifier = deployid,
      timestamp = locs_dt,
      location.long = x,
      location.lat = y,
      Argos.orientation = error_ellipse_orientation,
      Argos.semi.minor = error_semi_minor_axis,
      Argos.semi.major = error_semi_major_axis,
      Argos.location.class = quality
    )
  locs_df <- ctmm::as.telemetry(
    object = locs_df, 
    projection = out_proj)

  return(locs_df)
}

In [ ]:
pl_telem_spring <- as_telem(locs_sf_spring |> 
                              dplyr::filter(species == 'Spotted seal'),
                            out_proj = 'epsg:3571'
                            )

3383 ARGOS error ellipses missing. Using location class estimates from Vincent et al (2002).

Minimum sampling interval of 2 seconds in PL2005_5966_05L0002

Minimum sampling interval of 45 seconds in PL2005_5969_04L0021

Minimum sampling interval of 1.93 minutes in PL2005_5971_05L0017

Minimum sampling interval of 54 seconds in PL2006_5976_05L0245

...

Minimum sampling interval of 25 seconds in HF2010_2009_08L0055

Minimum sampling interval of 17 seconds in HF2010_2015_10A0217

Minimum sampling interval of 18 seconds in HF2010_2023_10A0229

Minimum sampling interval of 22 seconds in HF2014_1022_13A0223

Minimum sampling interval of 2 seconds in HF2022_1005_19U3070

In [ ]:
GUESS <- map(pl_telem_spring, ctmm.guess, interactive = FALSE)